###Google Collab Specs

In [ ]:
!df -h
!cat /proc/cpuinfo
!cat /proc/meminfo

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   30G   79G  28% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G 1005M  952M  52% /usr/sbin/docker-init
tmpfs           6.4G   40K  6.4G   1% /var/colab
/dev/sda1        68G   40G   29G  59% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology n

###Importing the libraries, importing the file, and reading the first 5 rows

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving glucose.csv to glucose.csv


In [ ]:
glucose_csv = 'glucose.csv'
df = pd.read_csv(glucose_csv)
df.head()

,Unnamed: 0,glucose,datetime
0,0,185.4,10/1/2014 19:14
1,1,178.2,10/1/2014 19:19
2,2,169.2,10/1/2014 19:23
3,3,176.4,10/1/2014 19:24
4,4,172.8,10/1/2014 19:29


###Testing the first 70% of the data and training the remaining 30%

#### Combined + by their own

In [ ]:
def train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Split the data into 70/30 train and test sets
    X_train_size = int(len(X) * 0.7)
    X_train, X_test, y_train, y_test = X[:-X_train_size], X[-X_train_size:], y[:-X_train_size], y[-X_train_size:]

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine predictions from all models
    combined_pred = (lin_pred + rand_pred + gb_pred + en_pred + svr_pred) / 5

    # Calculate root mean squared error (RMSE) for combined predictions
    rmse = sqrt(mean_squared_error(combined_pred, y_test))
    print('Mean Squared Error for Combined Model:', rmse)

    # Calculate individual RMSE for each model
    lin_rmse = sqrt(mean_squared_error(lin_pred, y_test))
    rand_rmse = sqrt(mean_squared_error(rand_pred, y_test))
    gb_rmse = sqrt(mean_squared_error(gb_pred, y_test))
    en_rmse = sqrt(mean_squared_error(en_pred, y_test))
    svr_rmse = sqrt(mean_squared_error(svr_pred, y_test))

    print('Mean Squared Error for Linear Regression Model:', lin_rmse)
    print('Mean Squared Error for Random Forest Regressor Model:', rand_rmse)
    print('Mean Squared Error for Gradient Boosting Model:', gb_rmse)
    print('Mean Squared Error for Elastic Net Regression Model:', en_rmse)
    print('Mean Squared Error for Support Vector Regression Model:', svr_rmse)

In [ ]:
train_models(glucose_csv)

Mean Squared Error for Combined Model: 35.32531403244956
Mean Squared Error for Linear Regression Model: 21.72898471770501
Mean Squared Error for Random Forest Regressor Model: 40.23996797356271
Mean Squared Error for Gradient Boosting Model: 40.26245949674234
Mean Squared Error for Elastic Net Regression Model: 21.733188666257107
Mean Squared Error for Support Vector Regression Model: 90.61370391972443


#### Aggregating the predictions from 2 machine learning models

In [ ]:
def train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Split the data into 70/30 train and test sets
    X_train_size = int(len(X) * 0.7)
    X_train, X_test, y_train, y_test = X[:-X_train_size], X[-X_train_size:], y[:-X_train_size], y[-X_train_size:]

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, gb_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, en_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, svr_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([rand_pred, gb_pred]), axis=0)
    aggregate_pred_6 = np.mean(np.array([rand_pred, en_pred]), axis=0)
    aggregate_pred_7 = np.mean(np.array([rand_pred, svr_pred]), axis=0)
    aggregate_pred_8 = np.mean(np.array([gb_pred, en_pred]), axis=0)
    aggregate_pred_9 = np.mean(np.array([gb_pred, svr_pred]), axis=0)
    aggregate_pred_10 = np.mean(np.array([en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4,
                       aggregate_pred_5, aggregate_pred_6, aggregate_pred_7, aggregate_pred_8,
                       aggregate_pred_9, aggregate_pred_10]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest Model is:', aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Gradient Boosting Model is:', aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Elastic Net Regression Model is:', aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Support Vector Regression Model is:', aggregate_rmse[3])
    print('Mean Squared Error for Random Forest + Gradient Boosting Model is:', aggregate_rmse[4])
    print('Mean Squared Error for Random Forest + Elastic Net Regression Model is:', aggregate_rmse[5])
    print('Mean Squared Error for Random Forest + Support Vector Regression Model is:', aggregate_rmse[6])
    print('Mean Squared Error for Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[7])
    print('Mean Squared Error for Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[8])
    print('Mean Squared Error for Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[9])

In [ ]:
train_models(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest Model is: 27.873842428497053
Mean Squared Error for Linear Regression + Gradient Boosting Model is: 27.90927726324782
Mean Squared Error for Linear Regression + Elastic Net Regression Model is: 21.73088240313973
Mean Squared Error for Linear Regression + Support Vector Regression Model is: 48.9673607268649
Mean Squared Error for Random Forest + Gradient Boosting Model is: 40.04531236624086
Mean Squared Error for Random Forest + Elastic Net Regression Model is: 27.892970164789148
Mean Squared Error for Random Forest + Support Vector Regression Model is: 59.71955566536697
Mean Squared Error for Gradient Boosting + Elastic Net Regression Model is: 27.92794602659255
Mean Squared Error for Gradient Boosting + Support Vector Regression Model is: 59.511815359191466
Mean Squared Error for Elastic Net Regression + Support Vector Regression Model is: 48.99347423254314


#### Aggregating the predictions from 3 machine learning models

In [ ]:
def train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Split the data into 70/30 train and test sets
    X_train_size = int(len(X) * 0.7)
    X_train, X_test, y_train, y_test = X[:-X_train_size], X[-X_train_size:], y[:-X_train_size], y[-X_train_size:]

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred, gb_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, rand_pred, en_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, rand_pred, svr_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([lin_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_6 = np.mean(np.array([lin_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_7 = np.mean(np.array([rand_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_8 = np.mean(np.array([rand_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_9 = np.mean(np.array([rand_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_10 = np.mean(np.array([gb_pred, en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4,
                       aggregate_pred_5, aggregate_pred_6, aggregate_pred_7, aggregate_pred_8,
                       aggregate_pred_9, aggregate_pred_10]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting Model is:', aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression Model is:', aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Random Forest + Support Vector Regression Model is:', aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[3])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[4])
    print('Mean Squared Error for Linear Regression + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[5])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[6])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[7])
    print('Mean Squared Error for Random Forest + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[8])
    print('Mean Squared Error for Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[9])

In [ ]:
train_models(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting Model is: 31.491716939677396
Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression Model is: 24.821731554935695
Mean Squared Error for Linear Regression + Random Forest + Support Vector Regression Model is: 43.02772776708157
Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression Model is: 24.852835285086094
Mean Squared Error for Linear Regression + Gradient Boosting + Support Vector Regression Model is: 42.90750724962869
Mean Squared Error for Linear Regression + Elastic Net Regression + Support Vector Regression Model is: 36.40246196081707
Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression Model is: 31.506274205321656
Mean Squared Error for Random Forest + Gradient Boosting + Support Vector Regression Model is: 50.985477398099476
Mean Squared Error for Random Forest + Elastic Net Regression + Support Vector Regression M

#### Aggregating the predictions from 4 machine learning models

In [ ]:
def train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Split the data into 70/30 train and test sets
    X_train_size = int(len(X) * 0.7)
    X_train, X_test, y_train, y_test = X[:-X_train_size], X[-X_train_size:], y[:-X_train_size], y[-X_train_size:]

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, rand_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, rand_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, gb_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([rand_pred, gb_pred, en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4, aggregate_pred_5]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Elastic Net Regression Model is:',
          aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Support Vector Regression Model is:',
          aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[3])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[4])

In [ ]:
train_models(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Elastic Net Regression Model is: 27.82680644917649
Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Support Vector Regression Model is: 40.93816499700691
Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression + Support Vector Regression Model is: 35.371242117167455
Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is: 35.29431134508305
Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is: 40.952134445382484


###Randomly dropping the data for training

#### Combined + by their own

In [ ]:
def randomized_train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Randomly drop 30% of the data for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

    # Initialize and fit models
    models = [
        ('Linear Regression', LinearRegression()),
        ('Random Forest', RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)),
        ('Gradient Boosting', GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=100, random_state=1)),
        ('Elastic Net Regression', ElasticNet()),
        ('Support Vector Regression', SVR(kernel='rbf', C=1e3, gamma=0.1))
    ]

    model_summaries = []

    for model_name, model in models:
        # Fit the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate error metrics
        rmse = sqrt(mean_squared_error(y_pred, y_test))

        # Create model summary
        summary_out = f"""
        Model: {model_name}
        Mean Squared Error: {rmse}
        """

        model_summaries.append(summary_out)

    # Combine predictions from all models
    aggregate_pred = np.mean([model.predict(X_test) for _, model in models], axis=0)
    aggregate_rmse = sqrt(mean_squared_error(aggregate_pred, y_test))

    # Create aggregate summary
    aggregate_summary = f"""
    Aggregate Prediction (Mean of All Models)
    Mean Squared Error: {aggregate_rmse}
    """

    model_summaries.append(aggregate_summary)

    for summary in model_summaries:
        print(summary)

In [ ]:
randomized_train_models(glucose_csv)


        Model: Linear Regression
        Mean Squared Error: 21.352112657618825
        

        Model: Random Forest
        Mean Squared Error: 17.15038116622321
        

        Model: Gradient Boosting
        Mean Squared Error: 19.829108409925038
        

        Model: Elastic Net Regression
        Mean Squared Error: 21.349202196381142
        

        Model: Support Vector Regression
        Mean Squared Error: 73.41537863515956
        

    Aggregate Prediction (Mean of All Models)
    Mean Squared Error: 24.202408221336444
    


#### Aggregating the predictions from 2 machine learning models

In [ ]:
def randomized_train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Randomly drop 30% of the data for training
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=100, random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train)  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions on the test set
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, gb_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, en_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, svr_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([rand_pred, gb_pred]), axis=0)
    aggregate_pred_6 = np.mean(np.array([rand_pred, en_pred]), axis=0)
    aggregate_pred_7 = np.mean(np.array([rand_pred, svr_pred]), axis=0)
    aggregate_pred_8 = np.mean(np.array([gb_pred, en_pred]), axis=0)
    aggregate_pred_9 = np.mean(np.array([gb_pred, svr_pred]), axis=0)
    aggregate_pred_10 = np.mean(np.array([en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4,
                       aggregate_pred_5, aggregate_pred_6, aggregate_pred_7, aggregate_pred_8,
                       aggregate_pred_9, aggregate_pred_10]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest Model is:', aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Gradient Boosting Model is:', aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Elastic Net Regression Model is:', aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Support Vector Regression Model is:', aggregate_rmse[3])
    print('Mean Squared Error for Random Forest + Gradient Boosting Model is:', aggregate_rmse[4])
    print('Mean Squared Error for Random Forest + Elastic Net Regression Model is:', aggregate_rmse[5])
    print('Mean Squared Error for Random Forest + Support Vector Regression Model is:', aggregate_rmse[6])
    print('Mean Squared Error for Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[7])
    print('Mean Squared Error for Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[8])
    print('Mean Squared Error for Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[9])

In [ ]:
randomized_train_models(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest Model is: 18.094488509057626
Mean Squared Error for Linear Regression + Gradient Boosting Model is: 19.875947544542502
Mean Squared Error for Linear Regression + Elastic Net Regression Model is: 21.350523923712398
Mean Squared Error for Linear Regression + Support Vector Regression Model is: 41.26835394004242
Mean Squared Error for Random Forest + Gradient Boosting Model is: 17.893883242804847
Mean Squared Error for Random Forest + Elastic Net Regression Model is: 18.099896972603958
Mean Squared Error for Random Forest + Support Vector Regression Model is: 40.09004535594504
Mean Squared Error for Gradient Boosting + Elastic Net Regression Model is: 19.883003858433312
Mean Squared Error for Gradient Boosting + Support Vector Regression Model is: 40.823514343175084
Mean Squared Error for Elastic Net Regression + Support Vector Regression Model is: 41.283491154792316


#### Aggregating the predictions from 3 machine learning models

In [ ]:
def randomized_train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Randomly drop 30% of the data for training
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred, gb_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, rand_pred, en_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, rand_pred, svr_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([lin_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_6 = np.mean(np.array([lin_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_7 = np.mean(np.array([rand_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_8 = np.mean(np.array([rand_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_9 = np.mean(np.array([rand_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_10 = np.mean(np.array([gb_pred, en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4,
                       aggregate_pred_5, aggregate_pred_6, aggregate_pred_7, aggregate_pred_8,
                       aggregate_pred_9, aggregate_pred_10]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting Model is:', aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression Model is:', aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Random Forest + Support Vector Regression Model is:', aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[3])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[4])
    print('Mean Squared Error for Linear Regression + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[5])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[6])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[7])
    print('Mean Squared Error for Random Forest + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[8])
    print('Mean Squared Error for Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[9])

In [ ]:
randomized_train_models(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting Model is: 18.340536831464863
Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression Model is: 18.966666382685677
Mean Squared Error for Linear Regression + Random Forest + Support Vector Regression Model is: 30.570376079235093
Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression Model is: 20.22109275671599
Mean Squared Error for Linear Regression + Gradient Boosting + Support Vector Regression Model is: 31.304658028879338
Mean Squared Error for Linear Regression + Elastic Net Regression + Support Vector Regression Model is: 31.887781068324113
Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression Model is: 18.3466834197497
Mean Squared Error for Random Forest + Gradient Boosting + Support Vector Regression Model is: 30.36573642058868
Mean Squared Error for Random Forest + Elastic Net Regression + Support Vector Regression Mo

#### Aggregating the predictions from 4 machine learning models

In [ ]:
def randomized_train_models(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Randomly drop 30% of the data for training
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, rand_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, rand_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, gb_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([rand_pred, gb_pred, en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4, aggregate_pred_5]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Elastic Net Regression Model is:',
          aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Support Vector Regression Model is:',
          aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[3])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[4])

In [ ]:
randomized_train_models(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Elastic Net Regression Model is: 18.854208518609745
Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Support Vector Regression Model is: 26.23439123941761
Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression + Support Vector Regression Model is: 26.587825222751732
Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is: 27.261872424228947
Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is: 26.24290946015178


###Dropping a data for testing from the data every 7 readings


#### Combined + by their own

In [ ]:
def train_model_7(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Split the data into 70/30 train and test sets
    X_train_size = int(len(X) * 0.7)
    X_train, y_train = X[:-X_train_size], y[:-X_train_size]

    # Drop every 7th row for the testing data
    X_test, y_test = X[-X_train_size:], y[-X_train_size:]
    drop_indices = np.arange(6, len(X_test), 7)
    X_test, y_test = np.delete(X_test, drop_indices, axis=0), np.delete(y_test, drop_indices)

    # Initialize models
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    lin_model = LinearRegression()
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    lin_model.fit(X_train, y_train)  # Linear Regression
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Calculate error metrics
    rmse_rand = sqrt(mean_squared_error(rand_pred, y_test))
    rmse_lin = sqrt(mean_squared_error(lin_pred, y_test))
    rmse_gb = sqrt(mean_squared_error(gb_pred, y_test))
    rmse_en = sqrt(mean_squared_error(en_pred, y_test))
    rmse_svr = sqrt(mean_squared_error(svr_pred, y_test))

    # Calculate additional metrics
    total_time = len(y_test)
    time_out_of_range = (y_test > 200).sum()
    percent_in_range = round(((total_time - time_out_of_range) / total_time) * 100)
    glucose_max = round(y_test.max())
    glucose_min = round(y_test.min())
    glucose_mean = round(y_test.mean())
    pred_out_of_range = ((rand_pred > 200) | (lin_pred > 200) | (gb_pred > 200) | (en_pred > 200) | (svr_pred > 200)).sum()
    pred_accuracy = round((pred_out_of_range / time_out_of_range) * 100)

    # Calculate and print the aggregate error metric
    aggregate_pred = (rand_pred + lin_pred + gb_pred + en_pred + svr_pred) / 5
    rmse_agg = sqrt(mean_squared_error(aggregate_pred, y_test))
    print('Mean Squared Error for Aggregate Model:', rmse_agg)

    # Print the error metrics for each model
    print('Mean Squared Error for Random Forest Model:', rmse_rand)
    print('Mean Squared Error for Linear Regression Model:', rmse_lin)
    print('Mean Squared Error for Gradient Boosting Model:', rmse_gb)
    print('Mean Squared Error for Elastic Net Regression Model:', rmse_en)
    print('Mean Squared Error for Support Vector Regression Model:', rmse_svr)

In [ ]:
train_model_7(glucose_csv)

Mean Squared Error for Aggregate Model: 35.09756173063176
Mean Squared Error for Random Forest Model: 39.93435450977156
Mean Squared Error for Linear Regression Model: 21.453010425392076
Mean Squared Error for Gradient Boosting Model: 39.961290442589764
Mean Squared Error for Elastic Net Regression Model: 21.456594205050283
Mean Squared Error for Support Vector Regression Model: 90.6422028179234


#### Aggregating the predictions from 2 machine learning models

In [ ]:
def train_models_7(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Drop every 7th row for the testing data
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in range(len(X)):
        if i % 7 == 0:
            X_test.append(X[i])
            y_test.append(y[i])
        else:
            X_train.append(X[i])
            y_train.append(y[i])
    X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, gb_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, en_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, svr_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([rand_pred, gb_pred]), axis=0)
    aggregate_pred_6 = np.mean(np.array([rand_pred, en_pred]), axis=0)
    aggregate_pred_7 = np.mean(np.array([rand_pred, svr_pred]), axis=0)
    aggregate_pred_8 = np.mean(np.array([gb_pred, en_pred]), axis=0)
    aggregate_pred_9 = np.mean(np.array([gb_pred, svr_pred]), axis=0)
    aggregate_pred_10 = np.mean(np.array([en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4,
                       aggregate_pred_5, aggregate_pred_6, aggregate_pred_7, aggregate_pred_8,
                       aggregate_pred_9, aggregate_pred_10]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest Model is:', aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Gradient Boosting Model is:', aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Elastic Net Regression Model is:', aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Support Vector Regression Model is:', aggregate_rmse[3])
    print('Mean Squared Error for Random Forest + Gradient Boosting Model is:', aggregate_rmse[4])
    print('Mean Squared Error for Random Forest + Elastic Net Regression Model is:', aggregate_rmse[5])
    print('Mean Squared Error for Random Forest + Support Vector Regression Model is:', aggregate_rmse[6])
    print('Mean Squared Error for Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[7])
    print('Mean Squared Error for Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[8])
    print('Mean Squared Error for Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[9])

In [ ]:
train_models_7(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest Model is: 16.760919770114718
Mean Squared Error for Linear Regression + Gradient Boosting Model is: 18.629762914528953
Mean Squared Error for Linear Regression + Elastic Net Regression Model is: 20.308352847345795
Mean Squared Error for Linear Regression + Support Vector Regression Model is: 36.878604965623175
Mean Squared Error for Random Forest + Gradient Boosting Model is: 16.71960191720922
Mean Squared Error for Random Forest + Elastic Net Regression Model is: 16.771590173554628
Mean Squared Error for Random Forest + Support Vector Regression Model is: 36.062248402065336
Mean Squared Error for Gradient Boosting + Elastic Net Regression Model is: 18.642449440131532
Mean Squared Error for Gradient Boosting + Support Vector Regression Model is: 36.860115588506915
Mean Squared Error for Elastic Net Regression + Support Vector Regression Model is: 36.89300331605874


#### Aggregating the predictions from 3 machine learning models

In [ ]:
def train_models_7(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Drop every 7th row for the testing data
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in range(len(X)):
        if i % 7 == 0:
            X_test.append(X[i])
            y_test.append(y[i])
        else:
            X_train.append(X[i])
            y_train.append(y[i])
    X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred, gb_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, rand_pred, en_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, rand_pred, svr_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([lin_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_6 = np.mean(np.array([lin_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_7 = np.mean(np.array([rand_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_8 = np.mean(np.array([rand_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_9 = np.mean(np.array([rand_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_10 = np.mean(np.array([gb_pred, en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4,
                       aggregate_pred_5, aggregate_pred_6, aggregate_pred_7, aggregate_pred_8,
                       aggregate_pred_9, aggregate_pred_10]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting Model is:', aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression Model is:', aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Random Forest + Support Vector Regression Model is:', aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[3])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[4])
    print('Mean Squared Error for Linear Regression + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[5])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression Model is:', aggregate_rmse[6])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Support Vector Regression Model is:', aggregate_rmse[7])
    print('Mean Squared Error for Random Forest + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[8])
    print('Mean Squared Error for Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:', aggregate_rmse[9])

In [ ]:
train_models_7(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting Model is: 17.06171193474637
Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression Model is: 17.698460148521423
Mean Squared Error for Linear Regression + Random Forest + Support Vector Regression Model is: 27.37317377724359
Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression Model is: 19.031791234401844
Mean Squared Error for Linear Regression + Gradient Boosting + Support Vector Regression Model is: 28.190007619392436
Mean Squared Error for Linear Regression + Elastic Net Regression + Support Vector Regression Model is: 28.585209382159416
Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression Model is: 17.071387392826157
Mean Squared Error for Random Forest + Gradient Boosting + Support Vector Regression Model is: 27.490374743141235
Mean Squared Error for Random Forest + Elastic Net Regression + Support Vector Regression 

#### Aggregating the predictions from 4 machine learning models

In [ ]:
def train_models_7(df_path):
    # Read the CSV file and drop the unnecessary column
    df = pd.read_csv(df_path)
    df = df.drop(columns=['Unnamed: 0'])

    # Define time windows and shifts
    window_interval = 30  # Time in minutes, smallest possible interval is 5 minutes
    frame_shift_1 = int(window_interval / 5)  # 6
    frame_shift_2 = int((window_interval * 2) / 5)  # 12
    frame_shift_3 = int((window_interval * 3) / 5)  # 18

    # Create columns for time windows by shifting the glucose column
    df[f'glucose_minus_{window_interval}'] = df['glucose'].shift(frame_shift_1)
    df[f'glucose_minus_{window_interval * 2}'] = df['glucose'].shift(frame_shift_2)
    df[f'glucose_minus_{window_interval * 3}'] = df['glucose'].shift(frame_shift_3)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Organize data
    x1, x2, x3, y = df[f'glucose_minus_{window_interval}'], df[f'glucose_minus_{window_interval * 2}'], \
        df[f'glucose_minus_{window_interval * 3}'], df['glucose']
    x1, x2, x3, y = np.array(x1), np.array(x2), np.array(x3), np.array(y)
    X = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1), x3.reshape(-1, 1)), axis=1)

    # Split the data into 70/30 train and test sets
    X_train_size = int(len(X) * 0.7)
    X_train, X_test, y_train, y_test = X[:-X_train_size], X[-X_train_size:], y[:-X_train_size], y[-X_train_size:]

    # Drop every 7th row for testing data
    X_test = X_test[::7]
    y_test = y_test[::7]

    # Initialize models
    lin_model = LinearRegression()
    rand_model = RandomForestRegressor(n_estimators=100, max_features=3, random_state=1)
    gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='squared_error', random_state=1)
    en_model = ElasticNet()
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)

    # Fit models
    lin_model.fit(X_train, y_train)  # Linear Regression
    rand_model.fit(X_train, y_train.ravel())  # Random Forest Regressor
    gb_model.fit(X_train, y_train)  # Gradient Boosting
    en_model.fit(X_train, y_train)  # Elastic Net Regression
    svr_model.fit(X_train, y_train)  # Support Vector Regression

    # Make predictions
    lin_pred = lin_model.predict(X_test)  # Linear Regression prediction
    rand_pred = rand_model.predict(X_test)  # Random Forest Regressor prediction
    gb_pred = gb_model.predict(X_test)  # Gradient Boosting prediction
    en_pred = en_model.predict(X_test)  # Elastic Net Regression prediction
    svr_pred = svr_model.predict(X_test)  # Support Vector Regression prediction

    # Combine Aggregate predictions by taking the mean of the models' predictions
    aggregate_pred_1 = np.mean(np.array([lin_pred, rand_pred, gb_pred, en_pred]), axis=0)
    aggregate_pred_2 = np.mean(np.array([lin_pred, rand_pred, gb_pred, svr_pred]), axis=0)
    aggregate_pred_3 = np.mean(np.array([lin_pred, rand_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_4 = np.mean(np.array([lin_pred, gb_pred, en_pred, svr_pred]), axis=0)
    aggregate_pred_5 = np.mean(np.array([rand_pred, gb_pred, en_pred, svr_pred]), axis=0)

    # Calculate accuracy and error metrics for each aggregate prediction
    aggregate_preds = [aggregate_pred_1, aggregate_pred_2, aggregate_pred_3, aggregate_pred_4, aggregate_pred_5]

    aggregate_rmse = []
    for pred in aggregate_preds:
        rmse = sqrt(mean_squared_error(pred, y_test))
        aggregate_rmse.append(rmse)

    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Elastic Net Regression Model is:',
          aggregate_rmse[0])
    print('Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Support Vector Regression Model is:',
          aggregate_rmse[1])
    print('Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[2])
    print('Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[3])
    print('Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is:',
          aggregate_rmse[4])

In [ ]:
train_models_7(glucose_csv)

Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Elastic Net Regression Model is: 27.862954911681083
Mean Squared Error for Linear Regression + Random Forest + Gradient Boosting + Support Vector Regression Model is: 40.918706941008374
Mean Squared Error for Linear Regression + Random Forest + Elastic Net Regression + Support Vector Regression Model is: 35.19609338860064
Mean Squared Error for Linear Regression + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is: 34.96752074189357
Mean Squared Error for Random Forest + Gradient Boosting + Elastic Net Regression + Support Vector Regression Model is: 40.93340262976634
